Ch.3 List & Tuple
============

-리스트는 동적 배열, 튜플은 정적 배열.

-책은 약간 C,C++스타일로 설명이 되어있음.

-리스트와 튜플안의 i번째 값을 '읽는' 동작은 O(1)의 시간이 걸린다

-해당 값을 찾는 작업을 해시를 이용하면 O(1), 정렬이 되어있는 리스트와 튜플의 경우 O(logN)의 시간이걸린다


In [3]:
%%timeit l = list(range(10))\
l[5]

38.7 ns ± 3.68 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [4]:
%%timeit l = list(range(10000000))
l[100000]

34.8 ns ± 2.19 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


* 리스트의 선형탐색

In [ ]:
def linear_search(needle,array):
    for i,item in enumerate(array):
        if item==needle:
            return i
    return -1

3.1 더 효율적인 탐색
==================

* 정렬되어있는 리스트에서 이진 탐색을 이용한 효율적인 탐색

    참고) 사전에서 탐색은 O(1)이 걸리나 리스트에서 사전으로 자료형을 바꾸는 데에 O(N)만큼 시간이 들어 무의미 

In [ ]:
def binary_search(needle, haystack):
    imin,imax = 0,len(haystack)
    while True:
        if imin > imax:
            return -1
        midpoint = (imin + imax) // 2
        if haystack[midpoint] > needle:
            imax = midpoint
        elif haystack[midpoint] < needle:
            imin = midpoint + 1                     # because of the cases such as imin=0,imax=1,needle=haystack[imax]
        else:
            return midpoint

+ 예시) bisect 모듈을 이용한 가까운 값 찾기

In [6]:
import bisect
import random


def find_closest(haystack, needle):
    # bisect.bisect_left will return the first value in the haystack
    # that is greater than the needle
    i = bisect.bisect_left(haystack, needle)
    if i == len(haystack):
        return i - 1
    elif haystack[i] == needle:
        return i
    elif i > 0:
        j = i - 1
        # since we know the value is larger than needle (and vice versa for the
        # value at j), we don't need to use absolute values here
        if haystack[i] - needle > needle - haystack[j]:
            return j
    return i


if __name__ == "__main__":
    important_numbers = []
    for i in range(10):
        new_number = random.randint(0, 1000)
        bisect.insort(important_numbers, new_number)

    # important_numbers will already be in order because we inserted new elements
    # with bisect.insort
    print(important_numbers)
    # > [14, 265, 496, 661, 683, 734, 881, 892, 973, 992]

    closest_index = find_closest(important_numbers, -250)
    print(f"Closest value to -250: {important_numbers[closest_index]}")
    # > Closest value to -250: 14

    closest_index = find_closest(important_numbers, 500)
    print(f"Closest value to 500: {important_numbers[closest_index]}")
    # > Closest value to 500: 496

    closest_index = find_closest(important_numbers, 1100)
    print(f"Closest value to 1100: {important_numbers[closest_index]}")
    # > Closest value to 1100: 992


[138, 142, 182, 356, 376, 456, 512, 516, 699, 776]
Closest value to -250: 138
Closest value to 500: 512
Closest value to 1100: 776


3.2 리스트와 튜플
===============

* 리스트:동적 / 튜플:정적
* 튜플은 파이썬 런타임에서 캐시하므로 사용할 때마다 커널에 메모리를 요청하지 않아도 된다.
* 크기가 N인 꽉 찬 리스트에 새로운 항목을 추가하면 여유분으로 M만큼 메모리를 할당한다. (M>N)

    -메모리 할당과 복사 요청 횟수를 줄이기 위함.
    
    -ex) N : 0 , 1-4 , 5-8 , 9-16 , ... , 991-1120  / M : 0 , 4   , 8   , 16   , ... , 1120

List
----

* 예제) append와 리스트 내포의 메모리 사용량과 실행 시간 차이 비교

In [16]:
import memory_profiler 
%load_ext memory_profiler

%memit [i*i for i in range(100000)]

The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler
peak memory: 61.61 MiB, increment: 3.50 MiB


In [17]:
%%memit l = []
for i in range(100000):
    l.append(i*2)

peak memory: 59.61 MiB, increment: 5.98 MiB


In [18]:
%%timeit l = []
for i in range(100000):
    l.append(i*2)

16.5 ms ± 1.26 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


Tuple
-----

- 파이썬은 GC를 통해 더는 사용되지 않는 변수에 할당된 메모리를 반환하지만 크기가 20이하인 튜플은 크기별로 최대 2만개까지 즉시 회수하지 않고 나중을 위해 저장해 둔다. -> 쉽고 빠르게 생성 가능
- 리스트와 다르가 크기가 1억이면 정확히 1억개 만큼의 메모리를 사용한다. (N,M)
- Good : 빠른생성, 메모리 부담 낮음
- Bad  : 내용변경 불가

Ch.4 Set & Dictionary
==================

- 삽입연산, search 모두 O(1).
- 메모리 사용량 많음

* 예시) 리스트와 셋에서 유일한 이름 찾기

In [2]:
import random
import string
import timeit


def list_unique_names(phonebook):
    unique_names = []
    for name, phonenumber in phonebook:
        first_name, last_name = name.split(" ", 1)
        for unique in unique_names:
            if unique == first_name:
                break
        else:
            unique_names.append(first_name)
    return len(unique_names)


def set_unique_names(phonebook):
    unique_names = set()
    for name, phonenumber in phonebook:
        first_name, last_name = name.split(" ", 1)
        unique_names.add(first_name)
    return len(unique_names)


def random_name():
    first_name = "".join(random.sample(string.ascii_letters, 8))
    last_name = "".join(random.sample(string.ascii_letters, 8))
    return "{} {}".format(first_name, last_name)


if __name__ == "__main__":
    phonebook = [("John Doe", "555-555-5555"), ("Albert Einstein", "212-555-5555")]

    print("Number of unique names from set method:", set_unique_names(phonebook))
    print("Number of unique names from list method:", list_unique_names(phonebook))

    setup = (
        "from __main__ import (large_phonebook, set_unique_names, list_unique_names)"
    )
    iterations = 50
    large_phonebook = [(random_name(), "555-555-5555") for i in range(1000)]

    t = timeit.timeit(
        stmt="list_unique_names(large_phonebook)", setup=setup, number=iterations
    )
    print(
        f"Finding unique names in a phonebook of length {len(large_phonebook)} "
        f"using lists took: {t / iterations:2e} seconds"
    )

    t = timeit.timeit(
        stmt="set_unique_names(large_phonebook)", setup=setup, number=iterations
    )
    print(
        f"Finding unique names in a phonebook of length {len(large_phonebook)} "
        f"using sets took: {t / iterations:2e} seconds"
    )


Number of unique names from set method: 2
Number of unique names from list method: 2
Finding unique names in a phonebook of length 1000 using lists took: 2.466297e-02 seconds
Finding unique names in a phonebook of length 1000 using sets took: 4.988023e-04 seconds


4.1 사전과 셋의 동작 원리
----------------------

* 예제 4-4 사전 탐색 과정

In [1]:
def index_sequence(key, mask=0b111, PERTURB_SHIFT=5):
    perturb = hash(key)
    i = perturb & mask
    yield i
    while True:
        perturb >>= PERTURB_SHIFT
        i = (i * 5 + perturb + 1) & mask
        yield i

* 예제 4-5 사용자 정의 해시 함수

In [6]:
class City(str):
    def __hash__(self):
        return ord(self[0])


if __name__ == "__main__":
    print(
        "Adding Rome, San Francisco, New York and Barcelona to a set.  New York and Barcenlona will collide!"
    )
    # We create a dictionary where we assign arbitrary values to cities
    data = {
        City("Rome"): "Italy",
        City("San Francisco"): "USA",
        City("New York"): "USA",
        City("Barcelona"): "Spain",
    }

Adding Rome, San Francisco, New York and Barcelona to a set.  New York and Barcenlona will collide!


* 사용자 정의 해시 예시 하나더

해석 : 해시를 직접 만들어 Point(1,1)를 아무리 여러개 만들어도 같은 '해시값'을 가지게 만들어 동일시 하게 해버린다.

In [15]:
class Point(object):
    def __init__(self, x, y):
        self.x, self.y = x, y


class PointHash(object):
    def __init__(self, x, y):
        self.x, self.y = x, y

    def __hash__(self):
        return hash((self.x, self.y))

    def __eq__(self, other):
        return self.x == other.x and self.y == other.y


if __name__ == "__main__":
    print("Test with default hash function")
    p1 = Point(1, 1)
    p2 = Point(1, 1)
    points = set([p1, p2])
    print("Contents of set([p1, p2]): ", points)
    print("Point(1, 1) in set([p1, p2]) = ", (Point(1, 1) in points))

    print("\nTest with custom hash function")
    p1 = PointHash(1, 1)
    p2 = PointHash(1, 1)
    points = set([p1, p2])
    print("Contents of set([p1, p2]): ", points)
    print("Point(1, 1) in set([p1, p2]) = ", (PointHash(1, 1) in points))


Test with default hash function
Contents of set([p1, p2]):  {<__main__.Point object at 0x7f0990582e50>, <__main__.Point object at 0x7f0990582b50>}
Point(1, 1) in set([p1, p2]) =  False

Test with custom hash function
Contents of set([p1, p2]):  {<__main__.PointHash object at 0x7f0990582710>}
Point(1, 1) in set([p1, p2]) =  True


In [8]:
ord("J") & 0b111

2

In [13]:
hash((0,1))

3713080549409410656

4.2 사전과 네임스페이스
=====================

- 변수,함수,모듈이 사용될 때 그 객체를 어디서 찾을지 결정하는 계층이 있다.
- 가장 먼저 지역 변수를 담은 locals()배열을 찾는다.
- 없다면 globals()사전에서 찾는다.
- 또 없다면 마지막으로 builtin 객체에서 찾는다.
- local --> global --> builtin
- 자세한 내용은 책 p.126참고

test3 , test2 ,test1 순으로 local , gloabl , builtin 이다.

In [16]:
import math
from math import sin


def test1(x):
    """
    >>> %timeit test1(123_456)
    162 µs ± 3.82 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
    """
    res = 1
    for _ in range(1000):
        res += math.sin(x)
    return res


def test2(x):
    """
    >>> %timeit test2(123_456)
    124 µs ± 6.77 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
    """
    res = 1
    for _ in range(1000):
        res += sin(x)
    return res


def test3(x, sin=math.sin):
    """
    >>> %timeit test3(123_456)
    105 µs ± 3.35 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
    """
    res = 1
    for _ in range(1000):
        res += sin(x)
    return res


In [18]:
import dis

dis.dis(test1)
print('-'*100)
dis.dis(test2)
print('-'*100)
dis.dis(test3)

 10           0 LOAD_CONST               1 (1)
              2 STORE_FAST               1 (res)

 11           4 SETUP_LOOP              30 (to 36)
              6 LOAD_GLOBAL              0 (range)
              8 LOAD_CONST               2 (1000)
             10 CALL_FUNCTION            1
             12 GET_ITER
        >>   14 FOR_ITER                18 (to 34)
             16 STORE_FAST               2 (_)

 12          18 LOAD_FAST                1 (res)
             20 LOAD_GLOBAL              1 (math)
             22 LOAD_METHOD              2 (sin)
             24 LOAD_FAST                0 (x)
             26 CALL_METHOD              1
             28 INPLACE_ADD
             30 STORE_FAST               1 (res)
             32 JUMP_ABSOLUTE           14
        >>   34 POP_BLOCK

 13     >>   36 LOAD_FAST                1 (res)
             38 RETURN_VALUE
----------------------------------------------------------------------------------------------------
 21           0 LOAD

Ch.5 이터레이터와 제너레이터
=========================

리스트는 무조건 전의 내용을 다 저장

이터레이터,제너레이터는 메모리 사용량이 '거의 없음'

(구체적으로 설명된게 잘없음 대부분 결과값을 하나씩 메모리에 저장해둔다는 말만 있음. 그러면 이전거는 낮은 메모리 계층에 저장해둔다는건가? 잘 모르겠음...)

피보나치 수 리스트를 여러 번 참조해야 한다면 리스트가 나을 수 있다.

CPU속도와 메모리 효율성 중 어느 쪽을 최적화할지 결정해야 한다.  ---> trade-off !!

일반적인 함수의 경우 return이나 assert등을 만나면 함수가 종료되어 버린다.

하지만 yield를 사용한 제너레이터로 만든 함수의 경우 '멈추었다가' 다시 call을 하면 그때부터 다시 돈다.

In [19]:
import timeit


def fibonacci_list(num_items):
    numbers = []
    a, b = 0, 1
    while len(numbers) < num_items:
        numbers.append(a)
        a, b = b, a + b
    return numbers


def fibonacci_gen(num_items):
    a, b = 0, 1
    while num_items:
        yield a
        a, b = b, a + b
        num_items -= 1

In [22]:
import timeit


def fibonacci_list(num_items):
    numbers = []
    a, b = 0, 1
    while len(numbers) < num_items:
        numbers.append(a)
        a, b = b, a + b
    return numbers


def fibonacci_gen(num_items):
    a, b = 0, 1
    while num_items:
        yield a
        a, b = b, a + b
        num_items -= 1


def test_fibonacci(func, N):
    for i in func(N):
        pass


if __name__ == "__main__":
    setup = "from __main__ import " "(test_fibonacci, fibonacci_gen, fibonacci_list, N)"
    iterations = 1000

    for N in (2, 100, 1_000, 100_00):
        t = timeit.timeit(
            stmt=f"test_fibonacci(fibonacci_list, N)", setup=setup, number=iterations
        )
        print(
            f"fibonacci_list took {t / iterations:.5e}s to calculate {N} fibonacci numbers"
        )

        t = timeit.timeit(
            stmt=f"test_fibonacci(fibonacci_gen, N)", setup=setup, number=iterations
        )
        print(
            f"fibonacci_gen took {t / iterations:.5e}s to calculate {N} fibonacci numbers"
        )


fibonacci_list took 1.04754e-06s to calculate 2 fibonacci numbers
fibonacci_gen took 5.43027e-07s to calculate 2 fibonacci numbers
fibonacci_list took 2.00265e-05s to calculate 100 fibonacci numbers
fibonacci_gen took 1.28990e-05s to calculate 100 fibonacci numbers
fibonacci_list took 2.18072e-04s to calculate 1000 fibonacci numbers
fibonacci_gen took 1.47779e-04s to calculate 1000 fibonacci numbers
fibonacci_list took 3.01583e-03s to calculate 10000 fibonacci numbers
fibonacci_gen took 1.89532e-03s to calculate 10000 fibonacci numbers


제너레이터를 사용하도록 코드를 만들때의 꿀팁

위의 내용의 경우 fibonacci_gen을 제너레이터로 사용하지만, 3으로 나뉘는 값을 배열에 저장하고 해당 배열에서 길이만 알나낸 다음에 데이터를 버린다. 이 과정에서 86MB 크기의 데이터를 소모한다.

아래 코드는 메모리 사용량이 거의 없다.

In [26]:
divisible_by_three = len([n for n in fibonacci_gen(10) if n%3 == 0])
divisible_by_three2 = sum(1 for n in fibonacci_gen(10) if n%3 == 0)

3

5.1 이터레이터로 무한급수 표현하기
-------------------------------

In [28]:
def fibonacci():
    i,j=0,1
    while True:
        yield j
        i,j = j,i+j
        
def fibonacci_naive():
    i,j=0,1
    count = 0
    while j< 5000:
        if j%2 :
            count += 1
        i,j = j,i+j
    return count

#이게 좋대
def fibonacci_transform():
    count=0
    for f in fibonacci():
        if f > 5000:
            break
        if f % 2:
            count += 1
    return count

from itertools import takewhile
def fibonacci_succint():
    first_5000 = takewhile(lambda x: x <= 5000, fibonacci())
    return sum(1 for x in first_5000 if x%2)

5.2 제네레이터의 지연 계산
-----------------------

In [31]:
from datetime import datetime
from itertools import count, filterfalse, groupby, islice
from random import normalvariate, randint

from scipy.stats import normaltest


def read_data(filename):
    with open(filename) as fd:
        for line in fd:
            data = line.strip().split(",")
            timestamp, value = map(int, data)
            yield datetime.fromtimestamp(timestamp), value


def read_fake_data(filename):
    for timestamp in count():
        #  We insert an anomalous data point approximately once a week
        if randint(0, 7 * 60 * 60 * 24 - 1) == 1:
            value = normalvariate(0, 1)
        else:
            value = 100
        yield datetime.fromtimestamp(timestamp), value


def groupby_day(iterable):
    key = lambda row: row[0].day
    for day, data_group in groupby(iterable, key):
        yield list(data_group)


def is_normal(data, threshold=1e-3):
    _, values = zip(*data)
    k2, p_value = normaltest(values)
    if p_value < threshold:
        return False
    return True


def filter_anomalous_groups(data):
    yield from filterfalse(is_normal, data)


def filter_anomalous_data(data):
    data_group = groupby_day(data)
    yield from filter_anomalous_groups(data_group)


if __name__ == "__main__":
    data = read_fake_data("fake_filename")
    anomaly_generator = filter_anomalous_data(data)
    first_five_anomalies = islice(anomaly_generator, 5)

    for data_anomaly in first_five_anomalies:
        start_date = data_anomaly[0][0]
        end_date = data_anomaly[-1][0]
        print(f"Anomaly from {start_date} - {end_date}")


Anomaly from 1970-01-01 09:00:00 - 1970-01-01 23:59:59
Anomaly from 1970-01-02 00:00:00 - 1970-01-02 23:59:59
Anomaly from 1970-01-03 00:00:00 - 1970-01-03 23:59:59
Anomaly from 1970-01-04 00:00:00 - 1970-01-04 23:59:59
Anomaly from 1970-01-05 00:00:00 - 1970-01-05 23:59:59
